In [2]:
import os
from collections import Counter
from IPython.display import Image, display
import numpy as np
import numpy.matlib

a = ["groom_list","lab_list","cinema_list","piano_list","barb_list",
 "military_list","none_list","vestment_list","curtain_list",
 "college_list","male_dress_list","female_dress_list"]

#Load Filenames and Predictions
filenames = open('filenames.csv').read().splitlines()
data = np.load('predictions.npy')
data.shape
predictions = data

model_dir = 'models/'
model_labels = os.path.join(model_dir, 'imagenet_comp_graph_label_strings.txt')
labels = []
with open(model_labels) as f:
    for label in f.readlines():
        labels.append(label.strip())
len(labels)

#create dictionary of filenames as keys and top 5 prediction labels as values
file_predictions = {}
for fn, pred in zip(filenames, data): 
    file_predictions[fn]=set()
    for ind in reversed(pred.argsort()[-5:]): #indicies of the top 5 predictions
        file_predictions[fn].add(labels[ind])

def show(fn):
    display(Image(filename='../Teenie_Harris_JPG640/' + fn))

#load the preselected partitions
partitions = {}
for tag_list in a:
    partitions[tag_list] = np.load(tag_list+".npy")
    print tag_list
    print len(partitions[tag_list])


groom_list
4873
lab_list
6830
cinema_list
2261
piano_list
5236
barb_list
5115
military_list
6148
none_list
12397
vestment_list
1789
curtain_list
2463
college_list
2272
male_dress_list
1760
female_dress_list
5387


In [19]:
#takes 2.25 minutes on the none_list
for label in partitions:
    image_names = partitions["female_dress_list"] 
    similarities = numpy.empty([len(image_names), len(image_names)])
    for i in xrange(len(image_names)): #i is the filename not the index
        for j in xrange(i+1): #j is the filename not the index
            if i > j:
                num_in_common = len(file_predictions[image_names[i]].intersection(file_predictions[image_names[j]]))
                similarities[i,j] = num_in_common
                similarities[j,i] = similarities[i,j]
            elif i == j:
                similarities[i,j] = 5
    print similarities[0]
    break
    

[ 5.  0.  1. ...,  2.  1.  0.]


In [72]:
print similarities.shape
for i in xrange(6):
    print i, len(np.where(similarities[120] == i)[0])
#     print np.where(similarities[9] == i)

(6830, 6830)
0 0
1 4824
2 1791
3 209
4 5
5 1


In [20]:
import matplotlib.pyplot as plt
try:
     from matplotlib.finance import quotes_historical_yahoo_ochl
except ImportError:
     # quotes_historical_yahoo_ochl was named quotes_historical_yahoo before matplotlib 1.4
     from matplotlib.finance import quotes_historical_yahoo as quotes_historical_yahoo_ochl
from matplotlib.collections import LineCollection
from sklearn import cluster, covariance, manifold

# documentation for the affinity propogation
# http://scikit-learn.org/stable/modules/generated/sklearn.cluster.affinity_propagation.html#sklearn.cluster.affinity_propagation
_, labels = cluster.affinity_propagation(similarities)
n_labels = labels.max()


In [21]:
import pickle

#These are the indicies of the cluster centers
with open('affinityPropResults_female_dress_list.pickle', 'wb') as handle:
    pickle.dump(_, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('labels_female_dress_list.pickle', 'wb') as handle:
    pickle.dump(labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)

# print a == b

In [22]:
print len(image_names)
print n_labels
n_labels = labels.max()

#note the biggest cluster in none_type is 47
#All the clusters that have more than phi photos
phi = 25
length_array = [0]*100
new_dict = {}
for i in range(n_labels + 1):
    if sum(labels == i) > (phi): #all the clusters that have more than phi elements
        print('Cluster %i: %s' % ((i + 1), ', '.join(image_names[labels == i])))
        print('Cluster %i: %s' % ((i + 1), 'length equals' + str(len(image_names[labels == i]))))
    term = "Cluster"+str(i+1)
    matches = image_names[labels == i]
    length_array[len(matches)]+=1
    new_dict[i+1] = matches
    
#         render the pictures out with imagemagick (montage maker)
#         output_fn = '"../{}-montage.jpg"'.format(term)
#         print 'montage -background White -geometry +0+0',' '.join(matches),output_fn,'&& open',output_fn

with open('ClusterPhotoMapping_female_dress_list.pickle', 'wb') as handle:
    pickle.dump(new_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

5387
1518
Cluster 451: 20124.jpeg, 21899.jpeg, 24208.jpeg, 25110.jpeg, 26044.jpeg, 26413.jpeg, 27965.jpeg, 28785.jpeg, 3059.jpeg, 30766.jpeg, 31316.jpeg, 31421.jpeg, 31448.jpeg, 33446.jpeg, 33646.jpeg, 33812.jpeg, 34218.jpeg, 36362.jpeg, 36365.jpeg, 36520.jpeg, 36714.jpeg, 36841.jpeg, 37404.jpeg, 37845.jpeg, 37847.jpeg, 38382.jpeg, 38907.jpeg, 38946.jpeg, 39866.jpeg, 39983.jpeg, 40756.jpeg, 40806.jpeg, 41073.jpeg, 43348.jpeg, 47360.jpeg, 48358.jpeg, 48436.jpeg, 48887.jpeg, 49914.jpeg, 50649.jpeg, 51047.jpeg, 52739.jpeg, 56909.jpeg, 58216.jpeg, 58953.jpeg, 61043.jpeg, 6318.jpeg, 63518.jpeg, 65509.jpeg, 65714.jpeg, 65727.jpeg, 65732.jpeg, 65879.jpeg, 66041.jpeg, 66308.jpeg, 66310.jpeg, 66573.jpeg, 66650.jpeg, 66796.jpeg, 66812.jpeg, 66815.jpeg, 66879.jpeg, 66904.jpeg, 66993.jpeg, 66994.jpeg, 67062.jpeg
Cluster 451: length equals66
Cluster 466: 21675.jpeg, 21843.jpeg, 22967.jpeg, 26876.jpeg, 28460.jpeg, 28534.jpeg, 33668.jpeg, 33731.jpeg, 33732.jpeg, 35227.jpeg, 36932.jpeg, 36934.jpeg, 37

In [18]:
print(length_array)
print(sum(length_array)) #1107 Clusters for 12397 photos. That's an average of 11 pictures per cluster

[0, 134, 135, 88, 68, 51, 41, 55, 37, 38, 37, 47, 32, 25, 26, 20, 18, 14, 16, 9, 6, 7, 2, 6, 3, 4, 4, 2, 1, 3, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
934


In [76]:
phi = 15
for i in range(n_labels + 1):
    if sum(labels == i) > (phi): #all the clusters that have more than phi elements
#         print('Cluster %i: %s' % ((i + 1), ', '.join(image_names[labels == i])))
        print('Cluster %i: %s' % ((i + 1), 'length equals ' + str(len(image_names[labels == i]))))
        term = "Cluster"+str(i+1)
        matches = image_names[labels == i]    
#         render the pictures out with imagemagick (montage maker)
        output_fn = '"../{}-montage.jpg"'.format(term)
        print 'montage -background White -geometry +0+0',' '.join(matches),output_fn,'&& open',output_fn

Cluster 1: length equals 17
montage -background White -geometry +0+0 11820.jpeg 13314.jpeg 19406.jpeg 20156.jpeg 22486.jpeg 23863.jpeg 32083.jpeg 34272.jpeg 43390.jpeg 46208.jpeg 53190.jpeg 55190.jpeg 59276.jpeg 65014.jpeg 69729.jpeg 70341.jpeg 70673.jpeg "../Cluster1-montage.jpg" && open "../Cluster1-montage.jpg"
Cluster 5: length equals 25
montage -background White -geometry +0+0 12414.jpeg 2108.jpeg 2128.jpeg 22084.jpeg 27488.jpeg 28666.jpeg 29830.jpeg 31628.jpeg 33943.jpeg 36694.jpeg 36695.jpeg 38114.jpeg 38116.jpeg 40757.jpeg 44323.jpeg 44665.jpeg 44666.jpeg 46948.jpeg 50524.jpeg 50824.jpeg 55351.jpeg 55905.jpeg 62919.jpeg 65134.jpeg 8256.jpeg "../Cluster5-montage.jpg" && open "../Cluster5-montage.jpg"
Cluster 6: length equals 21
montage -background White -geometry +0+0 12416.jpeg 17499.jpeg 19162.jpeg 27122.jpeg 30548.jpeg 36696.jpeg 3738.jpeg 3769.jpeg 42741.jpeg 44453.jpeg 4964.jpeg 59660.jpeg 60761.jpeg 61996.jpeg 62483.jpeg 69185.jpeg 70026.jpeg 70573.jpeg 71287.jpeg 72851.jp

In [26]:
with open('top5perphotoDictionary.pickle', 'wb') as handle:
    pickle.dump(file_predictions, handle, protocol=pickle.HIGHEST_PROTOCOL)